# Case class and Pattern matching
---

- an overview of companion object
- case class and pattern matching are twin constructs, and complement each other.
    - case classes come with a sensible default implementation of some properties which makes it an only candidate to define a non-encapsulated light-weight data constructors
    - pattern matching is similiar to switch statements but with more power, such as an inspection of a shape of given object etc..

### companion object
---

- an object is `companion object` when the name of a singleton object is the same as a class defined in the same source file.

In [1]:
// Person.scala

import scala.util.Random

class Person(val firstName: String, val lastName: String, val age: Int) {
    def name: String = firstName + " " + lastName
}

object Person {
    def mkId: String = Random.alphanumeric.take(20).mkString
}

import scala.util.Random


defined class Person
defined object Person

    - Scala has two namespaces, 1) for types and 2) for values, and depending on the position in expression, it determines whether we are referring to a class (type) or a companion object.

In [2]:
// it treats as a class on "type" position or next to `new` operator
val p: Person = new Person("John", "Doe", 21)

// it treats as a object on "value" position
Person.mkId

p: Person = ammonite.$sess.cmd0$Helper$Person@5c3591b2
res1_1: String = "3ct5igstJL3ll01Q18iz"

    - a class and its companion object can access each other's private members.
    - common use cases are, 
        1) auxiliary constructors 
        2) properties belong to a class but are independent of any particular object
    - The `apply` method in a singleton object receives special treatment, one can call it like a function application syntax.
        - Example: `ObjectName(param)` is similar to `ObjectName.apply(param)`
        - it allows to create auxiliary constructors for a class, and cerate an instances of it without using `new` operator.

In [3]:
// re-define above class
class Person(val firstName: String, val lastName: String, val age: Int) {
    def name: String = firstName + " " + lastName
}

object Person {
    // properties independent of any particular object
    val min = 18
    def isMinor(age: Int): Boolean = age < min
    // auxiliary constructors
    def apply(firstName: String, lastName: String): Person = 
        new Person(firstName, lastName, min)
    def apply(name: String, age: Int): Person = {
        val splits = name.split(' ')
        new Person(splits(0), splits(1), age)
    }
}

// Person("John", "Doe")  == Person.apply("John", "Doe")
// Person("John Doe", 12) == Person.apply("John Doe", 12)

// creating new instances without using `new` operator
val j1 = Person("John", "Doe")
val j2 = Person.apply("John Doe", 12)

Person.isMinor(j1.age)
Person.isMinor(j2.age)

defined class Person
defined object Person
j1: Person = ammonite.$sess.cmd2$Helper$Person@6c484b30
j2: Person = ammonite.$sess.cmd2$Helper$Person@68ffc788
res2_4: Boolean = false
res2_5: Boolean = true

### case class and case object
---

- Syntax: `case class <name>(<params>, ...)`

In [4]:
case class Calendar(date: String, time: String)

defined class Calendar

- Scala automatically generates a class and a companion object

In [5]:
// `typeOf` notebook only method

val cal = Calendar("05-04-2021", "07:30:00")
typeOf(cal).toString // Calendar
typeOf(Calendar).toString // Calendar.type

cal: Calendar = Calendar(date = "05-04-2021", time = "07:30:00")
res4_1: String = "cmd4.this.cmd3.Calendar"
res4_2: String = "cmd4.this.cmd3.Calendar.type"

    - constructor params will become class memebers but are immutable, so similar to as defining them as `val`.

In [5]:
cal.date = "05-05-2021" // error, fields are immutable

cmd5.sc:1: reassignment to val
val res5 = cal.date = "05-05-2021" // error, fields are immutable
                    ^Compilation Failed

: 

    - sensible implementations of `toString`, `equals` and `hashCode` methods.
    - a `copy` method, to create new instance with optional param to change value of any members

In [6]:
cal.toString // human readable toString
val calUpdated = cal.copy(date = "05-05-2021") // new object with updated value

res5_0: String = "Calendar(05-04-2021,07:30:00)"
calUpdated: Calendar = Calendar(date = "05-05-2021", time = "07:30:00")

- companion object created with a class has,
    - `apply` method with the same arguments as the class constructor
        - this allows use to create new instance of case class without invoking `new` operator
    - it also implements "extractor" to allow patter matching on case classes
- case object are useful when a case class has no constructor paramaters
    - it's same as a singleton object, and has sensible `toString` implementation

In [7]:
// case class NoParams() can be written as case object

case object NoParams

defined object NoParams

### pattern matching
---

Syntax:
```
<selector> match { 
  case <pattern> => <alternative>
  ...
}
``` 

In [8]:
Person.isMinor(j2.age) match {
    case true => "yes, minor!"
    case false => "no, adult!"
}

res7: String = "yes, minor!"

- it's an expression, just like conditional, so it returns a value
- kinds
    - wildcard pattern
        - as a default, catch-all alternative 
        - to ignore the parts of object that you do not care about
    - constant pattern
        - matches itself, any literal exression can be used as constant pattern
    - variable pattern
        - matches any object, but bind it to the name
            - a simple name starting with a lowercase letter is taken to be a pattern variable; all other references are taken to be constants.
    - constructor pattern (deep matches)
        - consists of a name (of a case class) and any nunber of pattern (any kind) in parentheses
        - one can write arbitrary nested object match, that's why it called deep matching

In [9]:
import scala.math.Pi

val cal = Calendar("05-04-2021", "07:30:00")

cal match {
    case Calendar(_, "07:30:00") => println("date is not important") // constructor match, wildcard ignore and constant match
    case Calendar(date, _) => println("date is " + date) // variable match
    case _ => println("match any calendar") // default
}

def isPi(n: Double): Boolean = n match {
    case Pi => true // constant match
    case _ => false
}

isPi(2.1)
isPi(Pi)

date is not important


import scala.math.Pi


cal: Calendar = Calendar(date = "05-04-2021", time = "07:30:00")
defined function isPi
res8_4: Boolean = false
res8_5: Boolean = true

    - sequence pattern
        - match sequence like data structure, Array or List, to match it's length or element at a position
        - `_*` in pattern matches any number of elements, including zero elements

In [10]:
def seqPattern(ls: List[Int]): Unit = ls match {
    case List(1, _, _) => println("first element is 1 from all three elements")
    case List(2, _*) => println("first element is 2 from all the elements")
    case _ => println("didn't match")
}

seqPattern(List(1, 2, 3))
seqPattern(List(2))
seqPattern(List(2, 1, 3, 5, 4))
seqPattern(List(10))

first element is 1 from all three elements
first element is 2 from all the elements
first element is 2 from all the elements
didn't match


defined function seqPattern

    - tuple pattern
        - matches elemenents enclosed in parantheses

In [11]:
val tup = (1, 3, 2)

tup match {
    case (_, 2, _) => println("middle element is 2")
    case (_, _, _) => println("tuple of three elements")
    case _ => println("didn't match")
}

tuple of three elements


tup: (Int, Int, Int) = (1, 3, 2)

    - typed pattern
        - to match the type of element and access it as a value of that type on the right hand side

In [12]:
def typedPattern(x: Any) = x match {
    case _: Int => println("int")
    case s: String => println(s.toUpperCase)
    case d: Double => println(d * d)
    case _ => println("type of " + x + " is not Int, String or Double.")
}

typedPattern(1.0)
typedPattern(1L)
typedPattern(10)
typedPattern("hello world")

1.0
type of 1 is not Int, String or Double.
int
HELLO WORLD


defined function typedPattern

- variable binding
- pattern guards
- pattern overlaps

In [13]:
case class Address(city: String, pincode: Int)
case class Person(name: String, address: Address)

def pincodeRange(pincode: Int): Boolean = Random.nextBoolean

def pattern(p: Person) = p match {
    case Person(name, Address("Mumbai", _)) if name.startsWith("A") => println("person " + name + " is from Mumbai.") // pattern guards
    case Person(_, addr @ Address(_, pincode)) if pincodeRange(pincode) => println("person's address " + addr + " is in range.") // variable binding
    case Person(name, Address(city, pincode)) => println("matches everything") // example of overlap
    case Person(name, Address("Bengaluru", _)) => println("unreachable block")
}

pattern(Person("Aron", Address("Bengaluru", 908912)))
pattern(Person("Aron", Address("Mumbai", 908912)))
pattern(Person("Ron", Address("Ahmedabad", 908912)))

person's address Address(Bengaluru,908912) is in range.
person Aron is from Mumbai.
person's address Address(Ahmedabad,908912) is in range.


defined class Address
defined class Person
defined function pincodeRange
defined function pattern

### exercise
---

1) Define a class `Person` with `firstName`, `lastName` and `age` as its members, and come up with a solution so that we can create an instance of `Person` class without using the `new` operator. 

2) What are the use cases of a companion object, and write down code as examples for each of those.

3) What happens when we define a companion object for a case class?

```
    case class MyInt(value: Int)
    object MyInt {
      def apply(a: Int, b: Int): MyInt = MyInt(a + b)
    }
```


4) How many types of patterns does Scala `match` expression support?

5) What is deep matching? How do you think it reduces the code length?